<a href="https://colab.research.google.com/github/maiterivas/Big-Data-Challenge/blob/main/digital_video_games_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3>Digital Video Games Amazon Reviews Data from: <br>
https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz </h3>

In [1]:
#Import Dependencies
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:13 http

In [2]:
#Connect to SQL
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-10-16 02:03:57--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K   654KB/s    in 1.4s    

2022-10-16 02:03:59 (654 KB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("digital_video_games_reviews").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"), sep="\t", header=True)

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|Digital_Video_Games|          2|            2|          3|   N|                N|A slight improvem...|I keep buying mad...| 2015-08-31|
|         US|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live Gift Card|Digital_Video_Games| 

In [4]:
#Print first 10 rows
df.show(10)

In [5]:
#Count the rows of the dataframe
df.count()

145431

In [6]:
#Print column names
df.columns

['marketplace',
 'customer_id',
 'review_id',
 'product_id',
 'product_parent',
 'product_title',
 'product_category',
 'star_rating',
 'helpful_votes',
 'total_votes',
 'vine',
 'verified_purchase',
 'review_headline',
 'review_body',
 'review_date']

In [7]:
#1 Schema uses review_id, customer_id, product_id, product_parent, review_date
#2 products use product_id, product_title
#3 customers use customer_id, customer_count(NEW column)
#4 Vine table uses review_id, star_rating, helpful_votes, total_votes, vine

In [8]:
#Drop unnecessary columns 
df = df.drop("product_category", "marketplace", "verified_purchase", "review_headline", "review_body")
df.show()

+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|review_date|
+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|          2|            2|          3|   N| 2015-08-31|
|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live Gift Card|          5|            0|          0|   N| 2015-08-31|
|   45765011| R3YOOS71KM5M9|B00DNHLFQA|     951665344|Command & Conquer...|          5|            0|          0|   N| 2015-08-31|
|     113118|R3R14UATT3OUFU|B004RMK5QG|     395682204|Playstation Plus ...|          5|            0|          0|   N| 2015-08-31|
|   22151364| RV2W9SGDNQA2C|B00G9BNLQE|     640460561|Saints Row IV - E...|        

In [9]:
#Drop null values
df = df.dropna(how="any")
df.show()

+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|review_date|
+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|          2|            2|          3|   N| 2015-08-31|
|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live Gift Card|          5|            0|          0|   N| 2015-08-31|
|   45765011| R3YOOS71KM5M9|B00DNHLFQA|     951665344|Command & Conquer...|          5|            0|          0|   N| 2015-08-31|
|     113118|R3R14UATT3OUFU|B004RMK5QG|     395682204|Playstation Plus ...|          5|            0|          0|   N| 2015-08-31|
|   22151364| RV2W9SGDNQA2C|B00G9BNLQE|     640460561|Saints Row IV - E...|        

In [10]:
#Count the rows of the dataframe - compare from the first
df.count()

145427

In [11]:
#Print Schema Types
df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- review_date: string (nullable = true)



In [12]:
#Match Schema value types to SQL Schema by casting them
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,BooleanType,DateType, IntegerType

df = df.withColumn("customer_id",col("customer_id").cast(IntegerType()))
df = df.withColumn("product_parent",col("product_parent").cast(IntegerType()))
df = df.withColumn("star_rating",col("star_rating").cast(IntegerType()))
df = df.withColumn("helpful_votes",col("helpful_votes").cast(IntegerType()))
df = df.withColumn("total_votes",col("total_votes").cast(IntegerType()))
df = df.withColumn("review_date",col("review_date").cast(DateType()))

In [13]:
#Drop null values again in case there are conflicts after casting columns
df = df.dropna(how="any")
df.show()

+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|review_date|
+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|          2|            2|          3|   N| 2015-08-31|
|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live Gift Card|          5|            0|          0|   N| 2015-08-31|
|   45765011| R3YOOS71KM5M9|B00DNHLFQA|     951665344|Command & Conquer...|          5|            0|          0|   N| 2015-08-31|
|     113118|R3R14UATT3OUFU|B004RMK5QG|     395682204|Playstation Plus ...|          5|            0|          0|   N| 2015-08-31|
|   22151364| RV2W9SGDNQA2C|B00G9BNLQE|     640460561|Saints Row IV - E...|        

In [14]:
#Count the rows of the dataframe
df.count()

145427

In [15]:
#Print Schema Types after casting columns
df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- review_date: date (nullable = true)



In [16]:
#Make review_id_table as df
review_id_table = df.select(["review_id","customer_id","product_id","product_parent","review_date"])
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RSH1OZ87OYK92|   21269168|B013PURRZW|     603406193| 2015-08-31|
|R1WFOQ3N9BO65I|     133437|B00F4CEHNK|     341969535| 2015-08-31|
| R3YOOS71KM5M9|   45765011|B00DNHLFQA|     951665344| 2015-08-31|
|R3R14UATT3OUFU|     113118|B004RMK5QG|     395682204| 2015-08-31|
| RV2W9SGDNQA2C|   22151364|B00G9BNLQE|     640460561| 2015-08-31|
|R3CFKLIZ0I2KOB|   22151364|B00IMIL498|     621922192| 2015-08-31|
|R1LRYU1V0T3O38|   38426028|B00S00IJH8|     215163395| 2015-08-31|
| R44QKV6FE5CJ2|    6057518|B004RMK4BC|     384246568| 2015-08-31|
|R2TX1KLPXXXNYS|   20715661|B00K59HKIQ|     384246568| 2015-08-31|
|R1JEEW4C6R89BA|   26540306|B00K59HKIQ|     384246568| 2015-08-31|
|R3B3UHK1FO0ERS|    8926809|B004774IPU|     151985175| 2015-08-31|
|R2GVSDHW513SS1|   31525534|B002LIT9EC|     695277014| 2015-08

In [17]:
#Print review_id_table schema
review_id_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [18]:
#Make products table
products_table = df.select(["product_id", "product_title"])
products_table.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B013PURRZW|Madden NFL 16 - X...|
|B00F4CEHNK| Xbox Live Gift Card|
|B00DNHLFQA|Command & Conquer...|
|B004RMK5QG|Playstation Plus ...|
|B00G9BNLQE|Saints Row IV - E...|
|B00IMIL498|Double Dragon: Ne...|
|B00S00IJH8|              Sims 4|
|B004RMK4BC|Playstation Netwo...|
|B00K59HKIQ|Playstation Netwo...|
|B00K59HKIQ|Playstation Netwo...|
|B004774IPU|Sid Meier's Civil...|
|B002LIT9EC|Build-a-lot 4: Po...|
|B008ALUBYQ|       Borderlands 2|
|B010KYDNDG|Minecraft for PC/...|
|B008ALUBYQ|       Borderlands 2|
|B004RMK4P8|Playstation Netwo...|
|B004RMK5QG|Playstation Plus ...|
|B00BI16Z22|Brink of Consciou...|
|B004VSTQ2A|Xbox Live Subscri...|
|B00GAC1D2G|Playstation Netwo...|
+----------+--------------------+
only showing top 20 rows



In [19]:
#Print products_table schema
products_table.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [20]:
#Make Customer table TWO since this is the second dataset -- must make customer_count column
customers_table = df.groupBy("customer_id").count()
customers_table = customers_table.withColumnRenamed("count","customer_count")
customers_table.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   26079415|             1|
|   12521601|             2|
|    4593696|             1|
|    1468259|             1|
|   38173350|             1|
|    3953141|             2|
|   20598424|             1|
|   46625306|             1|
|   47160828|             2|
|    1337034|             1|
|   14231016|             1|
|   49785748|             1|
|     712525|             1|
|   35339694|             1|
|    8211293|             1|
|   41832251|             1|
|    9037272|             1|
|   52036002|             1|
|   49514578|             1|
|   22875949|             1|
+-----------+--------------+
only showing top 20 rows



In [21]:
#Print customers_table schema
customers_table.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [22]:
#Convert new column (customer_count) to Integer
customers_table = customers_table.withColumn("customer_count",col("customer_count").cast(IntegerType()))

In [23]:
#Print customers_table schema after casting
customers_table.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [24]:
#Make VINE table
vine_table = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RSH1OZ87OYK92|          2|            2|          3|   N|
|R1WFOQ3N9BO65I|          5|            0|          0|   N|
| R3YOOS71KM5M9|          5|            0|          0|   N|
|R3R14UATT3OUFU|          5|            0|          0|   N|
| RV2W9SGDNQA2C|          5|            0|          0|   N|
|R3CFKLIZ0I2KOB|          5|            0|          0|   N|
|R1LRYU1V0T3O38|          4|            0|          0|   N|
| R44QKV6FE5CJ2|          5|            0|          0|   N|
|R2TX1KLPXXXNYS|          5|            0|          0|   N|
|R1JEEW4C6R89BA|          5|            0|          0|   N|
|R3B3UHK1FO0ERS|          1|            0|          0|   N|
|R2GVSDHW513SS1|          5|            0|          0|   N|
|R1R1NT516PYT73|          5|            0|          0|   N|
|R3K624QDQKENN9|          4|            

In [25]:
#Print vine_table schema
vine_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [30]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<AWS endpoint>:5432/postgres"
config = {"user":"postgres", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}


In [32]:
# Write Spark DataFrames to tables in RDS

review_id_table.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)
products_table.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)
customers_table.write.jdbc(url=jdbc_url, table="customers_two", mode=mode, properties=config)
vine_table.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)